In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.pyplot import figure
import time
import random
import pandas as pd

from skimage import measure

In [2]:
show_image = False
pxmm = 30 ## Used in kicad-parser (block 4)

In [3]:
def randomcolor():
    return (random.randrange(32,255,32),random.randrange(32,255,32),random.randrange(32,255,32))

In [4]:
def plotLineHigh(x0,y0, x1,y1,colorline):
    count = 0
    linesize = 0
    dx = x1 - x0
    dy = y1 - y0
    xi = 1
    if dx < 0:
        xi = -1
        dx = -dx
    
    D = 2*dx - dy
    x = x0
    for y in range(y0,y1):
        count = count + any(canvas[y,x]!=[0,0,0])*1
        linesize = linesize + 1
        if D > 0:
            x = x + xi
            D = D - 2*dy
        D = D + 2*dx
    return (count,linesize)

def plotLineLow(x0,y0, x1,y1,colorline):
    count = 0
    linesize = 0
    dx = x1 - x0
    dy = y1 - y0
    yi = 1
    if dy < 0:
        yi = -1
        dy = -dy
    
    D = 2*dy - dx
    y = y0

    for x in range(x0,x1):
        count = count + any(canvas[y,x]!=[0,0,0])*1
        linesize = linesize + 1
        if D > 0:
            y = y + yi
            D = D - 2*dx
        D = D + 2*dy        
    return (count,linesize)

def reline(x0,y0, x1,y1,color):
    if abs(y1 - y0) < abs(x1 - x0):
        if x0 > x1:
            (count,linesize) = plotLineLow(x1, y1, x0, y0,color)
        else:
            (count,linesize) = plotLineLow(x0, y0, x1, y1,color)        
    else:
        if y0 > y1:
            (count,linesize) = plotLineHigh(x1, y1, x0, y0,color)
        else:
            (count,linesize) = plotLineHigh(x0, y0, x1, y1,color)
    
    return (count,max(1,linesize))


In [6]:
im2 = cv2.imread('aout.png')
im = cv2.flip(im2,1)
imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [7]:
gcode = []
def g_print(gout):
    global gcode 
    
    gcode.append(gout)
    gcode.append("\n")

def g_write_to_file(filename):
    global gcode 
    
    f = open(filename,"wt")
    for stro in gcode:
        f.write(stro)
        
    f.close()
    
blockcounter =0

In [8]:
g_print("G21") # Metric
g_print("G90") # Absolute
g_print("G94") # Feed per minute

In [9]:
canvas = np.zeros([2636, 3568,3])
#canvas = cv2.drawContours(canvas, contours, -1, (255,0,0), 1)
cval = []

cv2.circle(canvas,(30,30),15,(255,0,0),-1)
cv2.circle(canvas,(60,30),15,(0,255,0),-1)
cv2.circle(canvas,(90,30),15,(0,0,255),-1)
cv2.circle(canvas,(120,30),15,(0,255,255),-1)
cv2.circle(canvas,(150,30),15,(255,255,0),-1)
cv2.circle(canvas,(180,30),15,(255,0,255),-1)
cv2.circle(canvas,(210,30),15,(255,255,255),-1)

contours.reverse()
for cc in contours:
    
    approx = cv2.approxPolyDP(cc, 3, True)
    [x1,y1]=approx[0][0]
    contourcolor=randomcolor()
    
    g_print("G0 Z2") ## dont need this in laser
    g_print("(Block-name: block "+str(blockcounter)+")\n(Block-expand: 0)\n(Block-enable: 1)")
    blockcounter = blockcounter + 1
    g_print("G01 F120") ## Feedrate 120 mm/min
    g_print("G0 X"+str(x1/pxmm)+" Y"+str(y1/pxmm))
    g_print("M04 S1000")
    
    
    for a in approx:
        [x,y]=a[0]
        g_print("G1 X"+str(x/pxmm)+" Y"+str(y/pxmm))
    
    g_print("M05\n ")


In [10]:
dfpd = pd.read_pickle("./padvia.pkl")
for index, row in dfpd.iterrows():
    ##The Hole
    
    x1 = (row['locxx']+row['drillx'])/pxmm
    x2 = row['locxx']/pxmm
    yy = (row['locy']/pxmm)
    r = (row['drillx']/pxmm)
    
    ## Goto rightmost
    g_print("(Block-name: blockvia "+str(blockcounter)+")\n(Block-expand: 0)\n(Block-enable: 1)")
    blockcounter = blockcounter + 1
    g_print("G01 F120") ## Feedrate 120 mm/min
    g_print("G0 X"+str(x1)+" Y"+str(yy))
    g_print("M04 S1000")
    
    
    ## Draw circle
    g_print("G02 X"+str(x1)+" I-"+str(r)+" F120")

g_print("M05")
g_print("G0 Z2")
g_print("G0 X0.5 Y0.5")

In [11]:
g_write_to_file("kic.gcode")